In [1]:
class Operation:
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes
        self.consumers = []
        for input_node in input_nodes:
            input_node.consumers.append(self)
        _default_graph.operations.append(self) # matmul을 예로 들자면, matmul이 operations 리스트에 추가되는 것. super()때문에 가능
    def compute(self):
        pass

In [2]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x,y])
    def compute(self, x_value, y_value):
        self.inputs = [x_value, y_value]
        return x_value + y_value

In [3]:
class matmul(Operation):
    def __init__(self, a, b):
        super().__init__([a,b])
    def compute(self, a_value, b_value):
        self.input = [a_value, b_value]
        return a_value.dot(b_value)

In [8]:
class placeholder:
    def __init__(self):
        self.consumers = []
        _default_graph.placeholders.append(self)

In [10]:
class Variable:
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.consumers = []
        _default_graph.variables.append(self)

In [6]:
class Graph:
    def __init__(self):
        self.operations = [] # 순서대로 matmul, add
        self.placeholders = [] # [1,2]
        self.variables = [] # 순서대로 [[1,0],[0,-1]], [1,1]
        
    def as_default(self):
        global _default_graph
        _default_graph = self

In [11]:
Graph().as_default() # 파이썬 전체에 Graph가 적용되어, class 바깥에서 변수를 만들어준 것과 같은 효과가 난다. 인상적.

A = Variable([[1,0],[0,-1]])
b = Variable([1,1])

x = placeholder()

y = matmul(A, x) # matmul이 먼저 operation으로 추가 된 다음

z = add(y, b) # add가 matmul 다음으로 operation에 추가된다.

In [18]:
import numpy as np

class Session:
    def run(self, operation, feed_dict = {}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node) == placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                
                node.output = node.compute(*node.inputs)
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        return operation.output
    
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation): # matmul, add는 모두 Operation 클래스를 상속받는다.
            # 반면 Variable이나 placeholder는 아무것도 상속받지 않는다. Operation클래스의 인스턴스 일리가 없다.
            for input_node in node.input_nodes:
                recurse(input_node) # add -> y(matmul) -> A(Variable) -> x(placeholder) -> b(Variable)
                # z -> b -> y -> x -> A
                
        nodes_postorder.append(node) # 따라서, Variable 또는 placeholder값은 node_postorder리스트에 추가된다.
        # A -> x -> y -> b -> z
        
            
            
    recurse(operation)
    return nodes_postorder

In [20]:
session = Session()
output = session.run(z, {
    x:[1,2]
})
print(output)

[ 2 -1]
